In [1]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import gc
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
warnings.simplefilter('ignore')

In [2]:
# dataの読み込み
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
submit_df = pd.read_csv('./submit_sample.csv',header=None)

In [3]:
# データの量の確認
train_df.shape,test_df.shape,submit_df.shape

((27100, 18), (18050, 17), (18050, 2))

In [4]:
# 訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df['y']=-999

In [5]:
# 訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df,test_df
gc.collect()

44

In [6]:
# カテゴリカラムの前処理
categorical_features = ['job', 'marital', 'education','default','housing','loan','contact','month','poutcome']
for col in categorical_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(all_df[col])
    lbl.transform(all_df[col])
    all_df[col]=lbl.transform(all_df[col])

In [7]:
#fe = ['age', 'marital', 'education', 'housing', 'loan', 'contact' ,'month', 'campaign', 'previous', 'poutcome']
#all_df['log_bal'] = np.log(all_df['balance'])
#all_df['bal_div_age'] = all_df['balance'] / all_df['age']
#all_df['bal_mul_age'] = all_df['balance'] * all_df['age']
#all_df['bal_div_age_p3'] = np.power(all_df['bal_div_age'], 3)
#sall_df['campaign_p2'] = np.power(all_df['campaign'], 2)
all_df['housing_and_loan'] = all_df['housing'] & all_df['loan']
all_df['month_q'] = all_df['month'] / 3



all_df.drop(['job', 'balance', 'day', 'duration', 'pdays'], axis=1)


,id,age,marital,education,default,housing,loan,contact,month,campaign,previous,poutcome,y,housing_and_loan,month_q
0,0,0.344444,1,1,0,1,0,0,8,3,0,1,0,0,2.666667
1,1,0.322222,2,2,0,0,0,0,1,2,0,3,1,0,0.333333
2,2,0.388889,1,2,0,1,0,0,8,1,0,0,0,0,2.666667
3,3,0.344444,1,1,0,1,1,2,7,2,0,0,0,1,2.333333
4,4,0.533333,1,0,0,1,0,1,0,1,0,3,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18045,18045,0.544444,1,2,0,1,0,0,4,2,0,0,-999,0,1.333333
18046,18046,0.377778,1,1,0,1,0,0,7,1,0,3,-999,0,2.333333
18047,18047,0.377778,2,1,0,1,0,2,7,2,0,3,-999,0,2.333333
18048,18048,0.344444,2,1,0,1,0,2,7,2,0,3,-999,0,2.333333


In [8]:
# 訓練データ、テストデータの分割
train_df = all_df[all_df['y']!=-999]
test_df = all_df[all_df['y']==-999]

In [9]:
y_train = train_df['y']
X_train = train_df.drop(['y','id'], axis=1)
X_test = test_df.drop(['y','id'], axis=1)

In [10]:
from sklearn.model_selection import KFold
from sklearn import metrics


y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)


params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)
    
    # eval
    #acc = accuracy_score(y_val, y_pred)
    fpr, tpr, thresholds = metrics.roc_curve(y_val, oof_train[valid_index])
    auc = metrics.auc(fpr, tpr)
    print('auc ', auc)

Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.228781	valid_1's binary_logloss: 0.215695
[20]	training's binary_logloss: 0.211043	valid_1's binary_logloss: 0.20317
[30]	training's binary_logloss: 0.200048	valid_1's binary_logloss: 0.19692
[40]	training's binary_logloss: 0.19163	valid_1's binary_logloss: 0.193064
[50]	training's binary_logloss: 0.184426	valid_1's binary_logloss: 0.190314
[60]	training's binary_logloss: 0.178445	valid_1's binary_logloss: 0.188777
[70]	training's binary_logloss: 0.172856	valid_1's binary_logloss: 0.187531
[80]	training's binary_logloss: 0.168055	valid_1's binary_logloss: 0.186941
[90]	training's binary_logloss: 0.163621	valid_1's binary_logloss: 0.18627
[100]	training's binary_logloss: 0.159368	valid_1's binary_logloss: 0.185428
[110]	training's binary_logloss: 0.15574	valid_1's binary_logloss: 0.185258
[120]	training's binary_logloss: 0.152098	valid_1's binary_logloss: 0.185056
[130]	training's binary_loglo

In [11]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.18479753709657076, 0.20904965476213458, 0.2071616492480855, 0.19989304566394617, 0.2029484250817564]
0.20077006237049871


In [12]:
from sklearn.metrics import accuracy_score


y_pred_oof = (oof_train > 0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

0.931549815498155

In [13]:
fpr, tpr, thresholds = metrics.roc_curve(y_train, oof_train)
metrics.auc(fpr, tpr)

# baseline is 0.8512576188085433
# 0.8517739935623644

0.8517739935623644

In [14]:
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_train, y_pred_oof)

In [15]:
cm

array([[24793,   195],
       [ 1660,   452]], dtype=int64)

## Reference
- CV
    - https://www.kaggle.com/sishihara/upura-kaggle-tutorial-06-validation
- Metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html#sklearn.metrics.auc